In [1]:
import numpy as np
from sklearn.cluster import KMeans
import sys
import cv2 as cv
import igraph as ig
import nibabel as nib
import os
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


In [2]:
class GaussianMixture:
    def __init__(self, X, n_components=5):
        self.n_components = n_components
        self.n_features = X.shape[1]
        self.n_samples = np.zeros(self.n_components)

        self.coefs = np.zeros(self.n_components)
        self.means = np.zeros((self.n_components, self.n_features))
        # Full covariance
        self.covariances = np.zeros(
            (self.n_components, self.n_features, self.n_features))

        label = KMeans(n_clusters=self.n_components, n_init=1).fit(X).labels_

        self.fit(X,label)
        
    def fit(self, X, label):
        uni_labels, count = np.unique(label, return_counts=True)
        self.n_samples[uni_labels] = count
        variance = 0.01
        for ci in uni_labels:
            n = self.n_samples[ci]
            self.coefs[ci] = n / np.sum(self.n_samples)
            self.means[ci] = np.mean(X[ci == label], axis=0)
            self.covariances[ci] = 0 if self.n_samples[ci] <= 1 else np.cov(
                X[ci == label].T)
            det = np.linalg.det(self.covariances[ci])
            if det <= 0:
                # Adds the white noise to avoid singular covariance matrix.
                self.covariances[ci] += np.eye(self.n_features) * variance
                det = np.linalg.det(self.covariances[ci])
        
        
    def calc_Score(self, X, ci):
        """Predict probabilities of samples belong to component ci
        ----------
        Args:
            X : array, shape (n_samples, n_features)
            ci : int       
        -------
        Returns:
            score : array, shape (n_samples,)
        """
        score = np.zeros(X.shape[0])
        if self.coefs[ci] > 0:
            diff = X - self.means[ci]
            mult = np.einsum('ij,ij->i', diff, np.dot(np.linalg.inv(self.covariances[ci]), diff.T).T)
            score = np.exp(-.5 * mult) / np.sqrt(2 * np.pi) / np.sqrt(np.linalg.det(self.covariances[ci]))
        return score

    def calc_Prob(self, X):
        """Predict probability (weighted score) of samples belong to the GMM
       
        ----------
        Args:
            X : array, shape (n_samples, n_features)
        
        -------
        Returns:
            prob : array, shape (n_samples,)
        """
        prob = [self.calc_Score(X, ci) for ci in range(self.n_components)]
        return np.dot(self.coefs, prob)

    def which_Component(self, X):
        """Predict samples belong to which GMM component
        ----------
        Args:
            X : array, shape (n_samples, n_features)
        -------
        Returns:
            comp : array, shape (n_samples,)
        """
        prob = np.array([self.calc_Score(X, ci)
                         for ci in range(self.n_components)]).T
        return np.argmax(prob, axis=1)



In [3]:
class GrabCut:
    def __init__(self, img, mask, rect=None, gmm_components=5):
        self.img = np.asarray(img, dtype=np.float64)
        self.rows, self.cols, _ = img.shape

        self.mask = mask
        if rect is not None:
            self.mask[rect[1]:rect[1] + rect[3],
                      rect[0]:rect[0] + rect[2]] = DRAW_PR_FG['val']
        self.bgd_indexes = np.where(np.logical_or(
            self.mask == DRAW_BG['val'], self.mask == DRAW_PR_BG['val']))
        self.fgd_indexes = np.where(np.logical_or(
            self.mask == DRAW_FG['val'], self.mask == DRAW_PR_FG['val']))
     
        self.gmm_components = gmm_components
        self.gamma = 50  #Chosen as such according to the original paper.
        self.beta = 0 #Will be changed later according to the original paper.

        self.left_V = np.empty((self.rows, self.cols - 1))
        self.upleft_V = np.empty((self.rows - 1, self.cols - 1))
        self.up_V = np.empty((self.rows - 1, self.cols))
        self.upright_V = np.empty((self.rows - 1, self.cols - 1))

        self.bgd_gmm = None #Will be changed later according to the original paper.
        self.fgd_gmm = None #Will be changed later according to the original paper.
        self.comp_idxs = np.empty((self.rows, self.cols), dtype=np.uint32)

        self.gc_graph = None
        self.gc_graph_capacity = None           # Edge capacities
        self.gc_source = self.cols * self.rows  # "object" terminal S
        self.gc_sink = self.gc_source + 1       # "background" terminal T

        
        left_difference = self.img[:, 1:] - self.img[:, :-1]  # Calculated for Smoothness term V and beta
        upleft_difference = self.img[1:, 1:] - self.img[:-1, :-1] # Calculated for Smoothness term V and beta
        up_difference = self.img[1:, :] - self.img[:-1, :] # Calculated for Smoothness term V and beta
        upright_difference = self.img[1:, :-1] - self.img[:-1, 1:] # Calculated for Smoothness term V and beta

        self.beta = np.sum(np.square(left_difference)) + np.sum(np.square(upleft_difference)) + \
            np.sum(np.square(up_difference)) + \
            np.sum(np.square(upright_difference))
        self.beta = 1 / (2 * self.beta / (
            # Each pixel has 4 neighbors (left, upleft, up, upright)
            4 * self.cols * self.rows
            
            - 3 * self.cols
            - 3 * self.rows  
            + 2))  # The first and last pixels in the 1st row are removed twice

        # Smoothness term V 
        self.left_V = self.gamma * np.exp(-self.beta * np.sum(
            np.square(left_difference), axis=2))
        self.upleft_V = self.gamma / np.sqrt(2) * np.exp(-self.beta * np.sum(
            np.square(upleft_difference), axis=2))
        self.up_V = self.gamma * np.exp(-self.beta * np.sum(
            np.square(up_difference), axis=2))
        self.upright_V = self.gamma / np.sqrt(2) * np.exp(-self.beta * np.sum(
            np.square(upright_difference), axis=2))

        self.bgd_gmm = GaussianMixture(self.img[self.bgd_indexes]) # Initialization of gaussian mixture for background 
        self.fgd_gmm = GaussianMixture(self.img[self.fgd_indexes]) # Initialization of gaussian mixture for foreground
 
        self.run()

    def run(self, num_iters=1, skip_learn_GMMs=False):
        for _ in range(num_iters):
            if not skip_learn_GMMs:
                self.assign_GMM_Parameter()
                self.learn_GMMs()
            self.Construct_Graph()
            self.Segmentation_Estimation()
            skip_learn_GMMs = False    
    
    def assign_GMM_Parameter(self):
        """Assign GMM components to pixels"""
        self.comp_idxs[self.bgd_indexes] = self.bgd_gmm.which_Component(
            self.img[self.bgd_indexes])
        self.comp_idxs[self.fgd_indexes] = self.fgd_gmm.which_Component(
            self.img[self.fgd_indexes])

    def learn_GMMs(self):
        """Learn GMM parameters from data z"""
        self.bgd_gmm.fit(self.img[self.bgd_indexes],
                         self.comp_idxs[self.bgd_indexes])
        self.fgd_gmm.fit(self.img[self.fgd_indexes],
                         self.comp_idxs[self.fgd_indexes])

    def Construct_Graph(self):
        bgd_indexes = np.where(self.mask.reshape(-1) == DRAW_BG['val'])
        fgd_indexes = np.where(self.mask.reshape(-1) == DRAW_FG['val'])
        pr_indexes = np.where(np.logical_or(
            self.mask.reshape(-1) == DRAW_PR_BG['val'], self.mask.reshape(-1) == DRAW_PR_FG['val']))

        edges = []
        self.gc_graph_capacity = []

        edges=self.Initiate_t_links(edges,bgd_indexes,fgd_indexes,pr_indexes)
        
        # n-links
        edges=self.Initiate_n_links(edges,bgd_indexes,fgd_indexes,pr_indexes)
    
    def Initiate_n_links(self,edges,bgd_indexes,fgd_indexes,pr_indexes):
        img_indexes = np.arange(self.rows * self.cols,
                                dtype=np.uint32).reshape(self.rows, self.cols)

        mask1 = img_indexes[:, 1:].reshape(-1)
        mask2 = img_indexes[:, :-1].reshape(-1)
        edges.extend(list(zip(mask1, mask2)))
        self.gc_graph_capacity.extend(self.left_V.reshape(-1).tolist())    

        mask1 = img_indexes[1:, 1:].reshape(-1)
        mask2 = img_indexes[:-1, :-1].reshape(-1)
        edges.extend(list(zip(mask1, mask2)))
        self.gc_graph_capacity.extend(
            self.upleft_V.reshape(-1).tolist())

        mask1 = img_indexes[1:, :].reshape(-1)
        mask2 = img_indexes[:-1, :].reshape(-1)
        edges.extend(list(zip(mask1, mask2)))
        self.gc_graph_capacity.extend(self.up_V.reshape(-1).tolist())

        mask1 = img_indexes[1:, :-1].reshape(-1)
        mask2 = img_indexes[:-1, 1:].reshape(-1)
        edges.extend(list(zip(mask1, mask2)))
        self.gc_graph_capacity.extend(
            self.upright_V.reshape(-1).tolist())
        
        self.gc_graph = ig.Graph(self.cols * self.rows + 2)
        self.gc_graph.add_edges(edges)
        return edges
    
    def Initiate_t_links(self,edges,bgd_indexes,fgd_indexes,pr_indexes):
         # t-links
        edges.extend(
            list(zip([self.gc_source] * pr_indexes[0].size, pr_indexes[0])))
        _D = -np.log(self.bgd_gmm.calc_Prob(self.img.reshape(-1, 3)[pr_indexes]))
        self.gc_graph_capacity.extend(_D.tolist())
        
        edges.extend(
            list(zip([self.gc_sink] * pr_indexes[0].size, pr_indexes[0])))
        _D = -np.log(self.fgd_gmm.calc_Prob(self.img.reshape(-1, 3)[pr_indexes]))
        self.gc_graph_capacity.extend(_D.tolist())

        edges.extend(
            list(zip([self.gc_source] * bgd_indexes[0].size, bgd_indexes[0])))
        self.gc_graph_capacity.extend([0] * bgd_indexes[0].size)
     
        edges.extend(
            list(zip([self.gc_sink] * bgd_indexes[0].size, bgd_indexes[0])))
        self.gc_graph_capacity.extend([9 * self.gamma] * bgd_indexes[0].size)   

        edges.extend(
            list(zip([self.gc_source] * fgd_indexes[0].size, fgd_indexes[0])))
        self.gc_graph_capacity.extend([9 * self.gamma] * fgd_indexes[0].size)
        
        edges.extend(
            list(zip([self.gc_sink] * fgd_indexes[0].size, fgd_indexes[0])))
        self.gc_graph_capacity.extend([0] * fgd_indexes[0].size)
        return edges

    def Segmentation_Estimation(self):
        """Estimate segmentation"""
        mincut = self.gc_graph.st_mincut(
            self.gc_source, self.gc_sink, self.gc_graph_capacity)
        pr_indexes = np.where(np.logical_or(
            self.mask == DRAW_PR_BG['val'], self.mask == DRAW_PR_FG['val']))
        img_indexes = np.arange(self.rows * self.cols,
                                dtype=np.uint32).reshape(self.rows, self.cols)
        self.mask[pr_indexes] = np.where(np.isin(img_indexes[pr_indexes], mincut.partition[0]),
                                         DRAW_PR_FG['val'], DRAW_PR_BG['val'])
        self.bgd_indexes = np.where(np.logical_or(
            self.mask == DRAW_BG['val'], self.mask == DRAW_PR_BG['val']))
        self.fgd_indexes = np.where(np.logical_or(
            self.mask == DRAW_FG['val'], self.mask == DRAW_PR_FG['val']))

In [4]:
def onMouse(event, x, y, flags, param):
    global img, img2, value, mask, rectangle, rect, rect_or_mask, ix, iy, rect_over, skip_learn_GMMs
    # From https://docs.opencv.org/4.x/db/d5b/tutorial_py_mouse_handling.html
    # Draw Rectangle
    if event == cv.EVENT_RBUTTONDOWN:
        rectangle = True
        ix, iy = x, y

    elif event == cv.EVENT_MOUSEMOVE:
        if rectangle == True:
            img = img2.copy()
            cv.rectangle(img, (ix, iy), (x, y), BLUE, 2)
            rect = (min(ix, x), min(iy, y), abs(ix-x), abs(iy-y))
            rect_or_mask = 0

    elif event == cv.EVENT_RBUTTONUP:
        rectangle = False
        rect_over = True
        cv.rectangle(img, (ix, iy), (x, y), BLUE, 2)
        rect = (min(ix, x), min(iy, y), abs(ix-x), abs(iy-y))
        rect_or_mask = 0



In [5]:
def Image_Loader(Image_Directory):
    os.chdir(Image_Directory)#Change accordingly
    img_path_index=[]
    label_path_index=[]
    print('Current working directory: ', os.getcwd()) #Check
    for datapath in range(len(os.listdir())):
            img = Image_Directory + '\\' + str(os.listdir()[datapath])
            img_path_index.append(img) 
            
    return img_path_index

Directory='C:\\Users\\ataka\\Desktop\\CS554 Final Project' #Change accordingly
os.chdir(Directory)
print('Check whether you are on the correct directory!!')
label_data_path=Image_Loader(Directory)
label_data_path

Check whether you are on the correct directory!!
Current working directory:  C:\Users\ataka\Desktop\CS554 Final Project


['C:\\Users\\ataka\\Desktop\\CS554 Final Project\\.ipynb_checkpoints',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\CRF_ALGO.drawio',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\CRF_ALGO.png',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\CRF_plane_145th_Subject92.png',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\CRF_plane_87th_Subject89.png',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\DenemeResmi.png',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\ExCRF.jpg',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\F1MAP.png',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\Final',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\GrabCut  Interactive Foreground Extraction using Iterated Graph Cuts.pdf',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\GrabCut.drawio',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\grabcut_outpu_for_plane_12th_Subject95.png',
 'C:\\Users\\ataka\\Desktop\\CS554 Final Project\\grabcut_outpu_for_plane_145th_Subject92.png',


In [6]:
BLUE = [255, 0, 0]        # rectangle color
RED = [0, 0, 255]         # PR BG
GREEN = [0, 255, 0]       # PR FG
BLACK = [0, 0, 0]         # sure BG
WHITE = [255, 255, 255]   # sure FG

DRAW_BG = {'color': BLACK, 'val': 0}
DRAW_FG = {'color': WHITE, 'val': 1}
DRAW_PR_FG = {'color': GREEN, 'val': 3}
DRAW_PR_BG = {'color': RED, 'val': 2}

# setting up flags
rect = (0, 0, 1, 1)
rectangle = False       # flag for drawing rect
rect_over = False       # flag to check if rect drawn
rect_or_mask = 100      # flag for selecting rect or mask mode
value = DRAW_FG         # drawing initialized to FG
thickness = 3           # brush thickness
skip_learn_GMMs = False # whether to skip learning GMM parameters

filename = 'plane_180th_Subject94.png'
LabelName='Labels_plane_180th_Subject94.png'

img = cv.imread(filename)
img2 = img.copy()                               # a copy of original image
mask = np.zeros(img.shape[:2], dtype=np.uint8)  # mask initialized to PR_BG
output = np.zeros(img.shape, np.uint8)           # output image to be shown
GroundTruth = cv.imread(LabelName)

# input and output windows
cv.namedWindow('output')
cv.namedWindow('input')
cv.setMouseCallback('input', onMouse)
cv.moveWindow('input', img.shape[1]+10, 90)

print(" Instructions: \n")
print(" Draw a rectangle around the object using right mouse button \n")

while(1):
    cv.imshow('output', output)
    cv.imshow('input', img)
    cv.imshow('GroundTruth', GroundTruth)
    k = cv.waitKey(1)
    # key bindings
    if k == 27:         # esc to exit
        break
        
    elif k == ord('r'):  # reset everything
        print("resetting \n")
        rect = (0, 0, 1, 1)
        rectangle = False
        rect_or_mask = 100
        rect_over = False
        value = DRAW_FG
        img = img2.copy()
        # mask initialized to PR_BG
        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        # output image to be shown
        output = np.zeros(img.shape, np.uint8)
    elif k == ord('n'):  # segment the image
        if (rect_or_mask == 0):         # grabcut with rect
            gc = GrabCut(img2, mask, rect)
            rect_or_mask = 1
        elif rect_or_mask == 1:         # grabcut with mask
            gc.run(skip_learn_GMMs=skip_learn_GMMs)
            skip_learn_GMMs = False
            
    elif k == ord('s'):  # save image
        bar = np.zeros((img.shape[0], 5, 3), np.uint8)
        res = np.hstack((img2, bar, img, bar, maskedOutput,bar,GroundTruth))
        FileName='grabcut_outpu_for_'+filename
        cv.imwrite(FileName, res)

        initial_segmentation_1d=GroundTruth[:,:,0].flatten()
        final_segmentation_1d=maskedOutput[:,:,0].flatten()
        # accuracy: (tp + tn) / (p + n)
        accuracy = accuracy_score(initial_segmentation_1d,final_segmentation_1d)
        print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(initial_segmentation_1d,final_segmentation_1d,pos_label=255)
        print('Precision: %f' % precision)
        # recall: tp / (tp + fn)
        recall = recall_score(initial_segmentation_1d,final_segmentation_1d,pos_label=255)
        print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(initial_segmentation_1d,final_segmentation_1d,pos_label=255)
        print('F1 score: %f' % f1)


    mask2 = np.where((mask == 1) + (mask == 3), 255, 0).astype('uint8')
    output = cv.bitwise_and(img2, img2, mask=mask2)
    output2=cv.threshold(output,0, 255,cv.THRESH_BINARY)
    maskedOutput=output2[1]
    
    

cv.destroyAllWindows()

 Instructions: 

 Draw a rectangle around the object using right mouse button 

